In [3]:
from extractors.pdf_extractor import extract_pdf_as_markdown,extract_docx_as_markdown
from chains.audit_chain import analyze_module
from chains.evidence_chain import analyze_evidence
from chains.improvement_chain import get_improvement_chain
from langchain_community.chat_models import ChatOpenAI
import pandas as pd
from dotenv import load_dotenv
import asyncio
import os
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

In [4]:
load_dotenv()
# print(os.getenv("OPENAI_API_KEY"))

file_path = "./files/"
controls = pd.read_json("ISO_27001_2022_Controls_List.json")


In [5]:
controls

,Section,Control ID,Control Title,Clause
0,A.5 (Organisational Controls),A.5.1,Policies for information security,"4,5"
1,A.5 (Organisational Controls),A.5.2,Information security roles and responsibilities,"4,5"
2,A.5 (Organisational Controls),A.5.3,Segregation of duties,5
3,A.5 (Organisational Controls),A.5.4,Management responsibilities,"5,6,9,10"
4,A.5 (Organisational Controls),A.5.5,Contact with authorities,
...,...,...,...,...
88,A.8 (Technological Controls),A.8.30,Outsourced development,
89,A.8 (Technological Controls),A.8.31,"Separation of development, test and production...",6
90,A.8 (Technological Controls),A.8.32,Change management,8
91,A.8 (Technological Controls),A.8.33,Test information,


In [6]:
import numpy as np

n = len(controls)
part_size = n // 5
remainder = n % 5

# Calculate split indices
sizes = [part_size + (1 if i < remainder else 0) for i in range(5)]
indices = np.cumsum([0] + sizes)

dfs = [controls.iloc[indices[i]:indices[i+1]].reset_index(drop=True) for i in range(5)]
df1, df2, df3, df4, df5 = dfs

In [7]:
def extract_file_as_markdown(file_path):
    ext = os.path.splitext(file_path)[1].lower()

    if ext == ".pdf":
        return extract_pdf_as_markdown(file_path)
    elif ext == ".docx":
        return extract_docx_as_markdown(file_path)
    else:
        raise ValueError(f"Unsupported file type: {ext}")

In [8]:
def read_folder_and_join_markdown(folder_path, file_list=None):
    """
    Args:
        folder_path (str): Root folder to search.
        file_list (list[str], optional): List of filenames (with or without relative paths) to include.
                                         If None, include all .pdf/.docx files.

    Returns:
        str: Combined markdown string from all valid files.
    """
    all_markdown = []
    normalized_file_list = set(os.path.normpath(f).lower() for f in file_list) if file_list else None
    print(f"Normalized file list: {normalized_file_list}")

    for root, _, files in os.walk(folder_path):
        for filename in files:
            ext = os.path.splitext(filename)[1].lower()
            if ext not in [".pdf", ".docx"]:
                continue

            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, folder_path)
            normalized_relative_path = os.path.normpath(relative_path).lower()

            if normalized_file_list and normalized_relative_path not in normalized_file_list:
                continue  # Skip files not in the list

            try:
                markdown = extract_file_as_markdown(file_path)
                all_markdown.append(f"## File: {relative_path}\n\n{markdown}")
            except Exception as e:
                print(f"❌ Error processing {file_path}: {e}")

    return "\n\n---\n\n".join(all_markdown)
def read_folder_and_join_markdown_exclude(folder_path, exclude_file_list=None):
    """
    Args:
        folder_path (str): Root folder to search.
        exclude_file_list (list[str], optional): List of file paths (relative to folder_path) to exclude.

    Returns:
        str: Combined markdown string from all valid, non-excluded files.
    """
    all_markdown = []
    normalized_exclude_list = set(os.path.normpath(f).lower() for f in exclude_file_list) if exclude_file_list else set()

    for root, _, files in os.walk(folder_path):
        for filename in files:
            if filename.startswith("._"):
                continue  
            ext = os.path.splitext(filename)[1].lower()
            if ext not in [".pdf", ".docx"]:
                continue

            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, folder_path)
            normalized_relative_path = os.path.normpath(relative_path).lower()

            if normalized_relative_path in normalized_exclude_list:
                continue  # Skip files in the exclusion list

            try:
                markdown = extract_file_as_markdown(file_path)
                all_markdown.append(f"## File: {relative_path}\n\n{markdown}")
            except Exception as e:
                print(f"❌ Error processing {file_path}: {e}")

    return "\n\n---\n\n".join(all_markdown)

In [9]:
file_list = ["Business Continuity Plan E-WISE [Restricted].docx.pdf","Information Security Policy [Internal].docx","Initial Implementation Plan - AI-Driven Document Compliance Analysis System (3).pdf"]
# file_list = ["Business Continuity Plan E-WISE [Restricted].docx.pdf","Information Security Policy [Internal].docx"]
evidence_text = read_folder_and_join_markdown_exclude(file_path, exclude_file_list=file_list)
text = read_folder_and_join_markdown(file_path,file_list=file_list)


✔️ Tesseract detected: tesseract v5.5.0.20241111
✔️ Tesseract detected: tesseract v5.5.0.20241111
✔️ Tesseract detected: tesseract v5.5.0.20241111
✔️ Tesseract detected: tesseract v5.5.0.20241111
✔️ Tesseract detected: tesseract v5.5.0.20241111
✔️ Tesseract detected: tesseract v5.5.0.20241111
✔️ Tesseract detected: tesseract v5.5.0.20241111
✔️ Tesseract detected: tesseract v5.5.0.20241111
✔️ Tesseract detected: tesseract v5.5.0.20241111
✔️ Tesseract detected: tesseract v5.5.0.20241111
✔️ Tesseract detected: tesseract v5.5.0.20241111
✔️ Tesseract detected: tesseract v5.5.0.20241111
Normalized file list: {'initial implementation plan - ai-driven document compliance analysis system (3).pdf', 'business continuity plan e-wise [restricted].docx.pdf', 'information security policy [internal].docx'}
✔️ Tesseract detected: tesseract v5.5.0.20241111
✔️ Tesseract detected: tesseract v5.5.0.20241111


In [12]:
len(text)

128612

In [ ]:

# llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3, api_key=os.getenv("OPENAI_API_KEY"))
# clauses = [df1,df2,df3,df4,df5]
# audits = []
# results = []
# for clause in clauses:
#     audit_chain = analyze_module(llm)
#     results.append(audit_chain.run(text=text, control_json=clause.to_json(orient='records', indent=2)))



In [13]:
from langchain.callbacks import get_openai_callback
import nest_asyncio
import sys
nest_asyncio.apply()

# text = extract_pdf_as_markdown(file_path)

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0,streaming=False, api_key=os.getenv("OPENAI_API_KEY"))
clauses = [df1,df2,df3,df4,df5]
results = []
total_tokens = 0
total_cost = 0.0

async def process_clause_async(clause, text, llm):
    def sync_callback_wrapper():
        with get_openai_callback() as cb:
            audit_chain = analyze_module(llm)
            result = audit_chain.run(
                text=text,
                control_json=clause.to_json(orient='records', indent=2)
            )
            return {
                "result": result,
                "tokens": cb.total_tokens,
                "cost": cb.total_cost,
                "controls": len(clause)
            }

    return await asyncio.to_thread(sync_callback_wrapper)

async def run_all_clauses():
    results = []
    total_tokens = 0
    total_cost = 0.0

    tasks = [process_clause_async(clause, text, llm) for clause in clauses]
    clause_results = await asyncio.gather(*tasks)

    for data in clause_results:
        results.append(data["result"])
        total_tokens += data["tokens"]
        total_cost += data["cost"]
        print(f"🔎 Clause Processed: {data['controls']} controls")
        print(f"🧠 Tokens used: {data['tokens']}")
        print(f"💵 Cost: ${data['cost']:.6f}")

    print("\n✅ All clauses processed (async).")
    print(f"🔢 Total tokens used: {total_tokens}")
    print(f"💰 Total cost: ${total_cost:.6f}")

    return results

if __name__ == "__main__":
    if sys.platform == "win32":
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    final_results = await run_all_clauses() if 'google.colab' in sys.modules or 'IPython' in sys.modules else asyncio.run(run_all_clauses())
    


C:\Users\Hp\AppData\Local\Temp\ipykernel_28412\2935787740.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0,streaming=False, api_key=os.getenv("OPENAI_API_KEY"))
C:\Users\Hp\AppData\Local\Temp\ipykernel_28412\2935787740.py:18: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = audit_chain.run(
c:\Users\Hp\Desktop\Codes\fiver\Document_Analyzer_Streamlit_App\chains\audit_chain.py:51: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prom

🔎 Clause Processed: 19 controls
🧠 Tokens used: 32582
💵 Cost: $0.017056
🔎 Clause Processed: 19 controls
🧠 Tokens used: 32216
💵 Cost: $0.016424
🔎 Clause Processed: 19 controls
🧠 Tokens used: 32093
💵 Cost: $0.016320
🔎 Clause Processed: 18 controls
🧠 Tokens used: 31859
💵 Cost: $0.016003
🔎 Clause Processed: 18 controls
🧠 Tokens used: 31778
💵 Cost: $0.015853

✅ All clauses processed (async).
🔢 Total tokens used: 160528
💰 Total cost: $0.081656


In [14]:
final_results[0]

'[\n  {\n    "Clause": "4",\n    "Section": "A.5 (Organisational Controls)",\n    "Control Id": "A.5.1",\n    "Control Title": "Policies for information security",\n    "Compliance": "✅",\n    "Policy": "Information Security Policy document defines baseline control measures, scope, roles, responsibilities, enforcement, and review processes for information security aligned with ISO 27001 standards.",\n    "Reference": "Information Security Policy [Internal].docx: \'This document details the policies and standards for the information security of Reducate... These policies will be based on ISO27001 standards.\'; \'This policy revision 1.0 will come into effect from 13-06-2022.\'; \'The director of Reducate is the owner of the security policy and any changes to the document have to be authorised by the director or their designate.\'",\n    "Gaps Identified": "",\n    "Recommended Action": ""\n  },\n  {\n    "Clause": "4,5",\n    "Section": "A.5 (Organisational Controls)",\n    "Control Id"

In [ ]:
evidence_chain = analyze_evidence(llm)
result = evidence_chain.run(
    text=evidence_text,
    control_json=final_results
)

ValueError: Missing some input keys: {'control_json'}

In [13]:
import json
results=result
with open("evidence_report.json", "w") as f:
    f.write(json.dumps(results, indent=2, ensure_ascii=False))

In [14]:
results

'[\n  {\n    "Clause": "4",\n    "Section": "A.5 (Organisational Controls)",\n    "Control Id": "A.5.1",\n    "Control Title": "Policies for information security",\n    "Compliance": "❌",\n    "Policy": "Information Security Policy document defines baseline control measures, scope, enforcement, roles and responsibilities, and management commitment to information security aligned with ISO 27001 standards.",\n    "Reference": "",\n    "Evidence Reference": "",\n    "Implemented Evidence": "",\n    "Evidence Recommendation": "Provide updated or additional evidence such as recent policy documents or management statements confirming current policy enforcement.",\n    "Gaps Identified": "No new evidence of updated or additional information security policies found in the provided text.",\n    "Recommended Action": "Submit current Information Security Policy documents or management communications to demonstrate ongoing policy enforcement."\n  },\n  {\n    "Clause": "4,5",\n    "Section": "A.5 

In [15]:
clean_json = results.strip()
if clean_json.startswith("```json"):
    clean_json = clean_json.removeprefix("```json").strip()
if clean_json.endswith("```"):
    clean_json = clean_json.removesuffix("```").strip()

# Step 2: Parse the cleaned JSON
try:
    parsed = json.loads(clean_json)
except json.JSONDecodeError as e:
    print("JSON parsing failed:", e)
    raise

# Step 3: Convert to DataFrame
resutls_df = pd.DataFrame(parsed)



In [18]:
resutls_df

,Clause,Section,Control Id,Control Title,Compliance,Policy,Reference,Evidence Reference,Implemented Evidence,Evidence Recommendation,Gaps Identified,Recommended Action
0,4,A.5 (Organisational Controls),A.5.1,Policies for information security,❌,Information Security Policy document defines b...,,,,Provide updated or additional evidence such as...,No new evidence of updated or additional infor...,Submit current Information Security Policy doc...
1,"4,5",A.5 (Organisational Controls),A.5.2,Information security roles and responsibilities,❌,Roles and responsibilities are clearly defined...,,,,"Provide organizational charts, role descriptio...",No new evidence found in the text regarding de...,Submit documentation or evidence showing curre...
2,5,A.5 (Organisational Controls),A.5.3,Segregation of duties,❌,Segregation of duties is mandated to reduce ri...,,,,Provide documented procedures or audit reports...,No new evidence found in the text regarding se...,"Submit evidence such as role assignments, proc..."
3,"5,6,9,10",A.5 (Organisational Controls),A.5.4,Management responsibilities,❌,Management is responsible for setting security...,,,,"Provide meeting minutes, management review rec...",No new evidence found in the text regarding ma...,"Submit evidence of management reviews, decisio..."
4,,A.5 (Organisational Controls),A.5.5,Contact with authorities,❌,Procedures and contacts for communication with...,,,,Provide incident response plans or contact lis...,No new evidence found in the text regarding co...,Submit documented procedures or contact lists ...
...,...,...,...,...,...,...,...,...,...,...,...,...
88,,A.8 (Technological Controls),A.8.30,Outsourced development,❌,Outsourced development requires review of SDLC...,,,,Provide outsourced development contracts and S...,No new evidence found in the text regarding ou...,Submit outsourced development documentation.
89,6,A.8 (Technological Controls),A.8.31,"Separation of development, test and production...",❌,"Development, test, and production environments...",,,,Provide environment separation policies and ac...,No new evidence found in the text regarding en...,Submit environment separation documentation.
90,8,A.8 (Technological Controls),A.8.32,Change management,❌,Changes to information processing facilities a...,,,,Provide change management procedures and records.,No new evidence found in the text regarding ch...,Submit change management documentation.
91,,A.8 (Technological Controls),A.8.33,Test information,❌,Test environments use limited or anonymised da...,,,,Provide test data management policies and evid...,No new evidence found in the text regarding te...,Submit test data management documentation.


In [19]:
resutls_df.to_excel("result.xlsx")

In [20]:
with open("output.md", "w", encoding="utf-8") as f:
    f.write(text)

print("✅ PDF content extracted and saved as 'output.md'")

✅ PDF content extracted and saved as 'output.md'
